In [1]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

## Create dataframe
raw = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep = ",", header = True)

In [7]:
# Display dataframe
raw.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [8]:
# Display schema
raw.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [9]:
# Create a temporary view of the DataFrame.
raw.createOrReplaceTempView('home_data')

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price = """ 
    SELECT EXTRACT(YEAR FROM date) AS year,
    ROUND(AVG(price), 2) AS avg_sale_price
    FROM home_data
    WHERE bedrooms == 4
    GROUP BY year
    ORDER BY year
    """
spark.sql(avg_price).show()

+----+--------------+
|year|avg_sale_price|
+----+--------------+
|2019|      300263.7|
|2020|     298353.78|
|2021|     301819.44|
|2022|     296363.88|
+----+--------------+



In [12]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_bybuilt = """ 
    SELECT date_built, 
    ROUND(AVG(price), 2) AS average_price
    FROM home_data
    WHERE bedrooms == 3 AND bathrooms == 3
    GROUP BY date_built
    ORDER BY date_built
    """
spark.sql(avg_price_bybuilt).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [13]:
# 5. What is the average price of a home for each year built that have 3 BR, 3 BA, 2 Floors 
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg_price_3BR3BAsqft = """ 
    SELECT date_built, 
    ROUND(AVG(price), 2) AS average_price 
    FROM home_data
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
    """
spark.sql(avg_price_3BR3BAsqft).show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [14]:
# 6. What is the "view" rating for the avg price of a home where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating = """ 
    SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_data
    WHERE price >= 350000
    """
spark.sql(view_rating).show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.2726294994354248 seconds ---


In [15]:
# 7. Cache the the temporary table home_data.
spark.sql("cache table home_data")

DataFrame[]

In [16]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_data')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 


In [ ]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

